# Module 01: LangGraph Fundamentals - Practice Notebook

**Level:** Beginner  
**Duration:** 3-4 hours  
**Updated:** December 2025 - Latest v1.0 patterns

## Learning Objectives

Master LangGraph basics:
- ✅ Build your first StateGraph
- ✅ Create nodes and edges
- ✅ Use START and END constants
- ✅ Implement conditional routing
- ✅ Work with state management


In [ ]:
# Setup - Install dependencies
%pip install -q -U langgraph langchain langchain-openai python-dotenv

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

print('✅ LangGraph ready!')


---

## Exercise 1: Your First Graph 🎯

**Objective:** Build a simple 2-node graph.

### Task
Create a graph with:
1. A state with a `message` field
2. Node 1: Uppercases the message
3. Node 2: Adds exclamation mark
4. Flow: START → Node1 → Node2 → END


In [ ]:
# Exercise 1: Build your first graph

class SimpleState(TypedDict):
    message: str

def uppercase_node(state: SimpleState):
    """Convert message to uppercase."""
    return {'message': state['message'].upper()}

def exclaim_node(state: SimpleState):
    """Add exclamation mark."""
    return {'message': state['message'] + '!'}

# Build the graph
workflow = StateGraph(SimpleState)
workflow.add_node('uppercase', uppercase_node)
workflow.add_node('exclaim', exclaim_node)

# Define the flow
workflow.add_edge(START, 'uppercase')
workflow.add_edge('uppercase', 'exclaim')
workflow.add_edge('exclaim', END)

# Compile and test
app = workflow.compile()
result = app.invoke({'message': 'hello world'})
print(f"Result: {result['message']}")  # Expected: HELLO WORLD!


---

## Exercise 2: Conditional Routing 🎯

**Objective:** Learn conditional edges for dynamic routing.

### Task
Create a sentiment analyzer that routes positive/negative messages differently.


In [ ]:
# Exercise 2: Conditional routing

class SentimentState(TypedDict):
    text: str
    sentiment: str
    response: str

def analyze_sentiment(state: SentimentState):
    """Simple sentiment detection."""
    positive_words = ['good', 'great', 'happy', 'love', 'excellent']
    negative_words = ['bad', 'sad', 'hate', 'terrible', 'awful']
    
    text_lower = state['text'].lower()
    
    if any(word in text_lower for word in positive_words):
        return {'sentiment': 'positive'}
    elif any(word in text_lower for word in negative_words):
        return {'sentiment': 'negative'}
    return {'sentiment': 'neutral'}

def positive_response(state: SentimentState):
    return {'response': 'Glad to hear positive feedback!'}

def negative_response(state: SentimentState):
    return {'response': 'Sorry to hear that. How can we improve?'}

def neutral_response(state: SentimentState):
    return {'response': 'Thanks for your input!'}

# Router function
def route_sentiment(state: SentimentState) -> str:
    return state['sentiment']

# Build graph with conditional routing
workflow = StateGraph(SentimentState)
workflow.add_node('analyze', analyze_sentiment)
workflow.add_node('positive', positive_response)
workflow.add_node('negative', negative_response)
workflow.add_node('neutral', neutral_response)

workflow.add_edge(START, 'analyze')
workflow.add_conditional_edges(
    'analyze',
    route_sentiment,
    {
        'positive': 'positive',
        'negative': 'negative',
        'neutral': 'neutral'
    }
)
workflow.add_edge('positive', END)
workflow.add_edge('negative', END)
workflow.add_edge('neutral', END)

app = workflow.compile()

# Test cases
test_cases = [
    'This is great!',
    'This is terrible.',
    'This is okay.'
]

for text in test_cases:
    result = app.invoke({'text': text, 'sentiment': '', 'response': ''})
    print(f"Text: '{text}'")
    print(f"Sentiment: {result['sentiment']}")
    print(f"Response: {result['response']}\n")


---

## Exercise 3: State with Reducers 🎯

**Objective:** Use reducers for accumulating state.

### Task
Build a multi-step processor that accumulates logs.


In [ ]:
# Exercise 3: State with reducers

def accumulate_logs(existing: list, new: list) -> list:
    """Append new logs to existing."""
    return existing + new

class ProcessState(TypedDict):
    data: str
    logs: Annotated[list, accumulate_logs]
    result: str

def step1(state: ProcessState):
    return {
        'data': state['data'].strip(),
        'logs': ['Step 1: Trimmed whitespace']
    }

def step2(state: ProcessState):
    return {
        'data': state['data'].lower(),
        'logs': ['Step 2: Converted to lowercase']
    }

def step3(state: ProcessState):
    return {
        'result': f"Processed: {state['data']}",
        'logs': ['Step 3: Added prefix']
    }

# Build pipeline
workflow = StateGraph(ProcessState)
workflow.add_node('step1', step1)
workflow.add_node('step2', step2)
workflow.add_node('step3', step3)

workflow.add_edge(START, 'step1')
workflow.add_edge('step1', 'step2')
workflow.add_edge('step2', 'step3')
workflow.add_edge('step3', END)

app = workflow.compile()

result = app.invoke({'data': '  HELLO WORLD  ', 'logs': [], 'result': ''})
print(f"Result: {result['result']}")
print(f"\nLogs:")
for log in result['logs']:
    print(f"  - {log}")


---

## Exercise 4: Cyclic Graph (Loop) 🎯

**Objective:** Create a graph with a loop for iterative processing.

### Task
Build a counter that increments until reaching a limit.


In [ ]:
# Exercise 4: Cyclic graph with loop

class CounterState(TypedDict):
    count: int
    max_count: int
    iterations: list

def increment(state: CounterState):
    new_count = state['count'] + 1
    return {
        'count': new_count,
        'iterations': state['iterations'] + [f"Count: {new_count}"]
    }

def should_continue(state: CounterState) -> str:
    """Decide whether to continue or end."""
    if state['count'] < state['max_count']:
        return 'continue'
    return 'end'

# Build graph with loop
workflow = StateGraph(CounterState)
workflow.add_node('increment', increment)

workflow.add_edge(START, 'increment')
workflow.add_conditional_edges(
    'increment',
    should_continue,
    {
        'continue': 'increment',  # Loop back!
        'end': END
    }
)

app = workflow.compile()

result = app.invoke({'count': 0, 'max_count': 5, 'iterations': []})
print(f"Final count: {result['count']}")
print(f"\nIterations:")
for iteration in result['iterations']:
    print(f"  {iteration}")


---

## Exercise 5: Streaming Execution 🎯

**Objective:** Use streaming to see intermediate results.

### Task
Build a multi-step workflow and stream the outputs.


In [ ]:
# Exercise 5: Streaming execution

class StreamState(TypedDict):
    input: str
    step1_output: str
    step2_output: str
    final_output: str

def process_step1(state: StreamState):
    import time
    time.sleep(0.5)  # Simulate work
    return {'step1_output': f"Step1: {state['input'].upper()}"}

def process_step2(state: StreamState):
    import time
    time.sleep(0.5)  # Simulate work
    return {'step2_output': f"Step2: {state['step1_output']}!!!"}

def finalize(state: StreamState):
    return {'final_output': f"Final: {state['step2_output']}"}

# Build workflow
workflow = StateGraph(StreamState)
workflow.add_node('step1', process_step1)
workflow.add_node('step2', process_step2)
workflow.add_node('finalize', finalize)

workflow.add_edge(START, 'step1')
workflow.add_edge('step1', 'step2')
workflow.add_edge('step2', 'finalize')
workflow.add_edge('finalize', END)

app = workflow.compile()

# Regular invoke (blocking)
print("=== Regular Invoke ===")
result = app.invoke({'input': 'hello', 'step1_output': '', 'step2_output': '', 'final_output': ''})
print(f"Result: {result['final_output']}")

# Streaming (see intermediate outputs)
print("\n=== Streaming ===")
for chunk in app.stream({'input': 'world', 'step1_output': '', 'step2_output': '', 'final_output': ''}):
    print(f"Chunk: {chunk}")


---

## 📚 Summary

You've mastered:
- ✅ Building basic StateGraphs
- ✅ Using START and END constants
- ✅ Conditional routing
- ✅ State reducers
- ✅ Cyclic graphs (loops)
- ✅ Streaming execution

**Next:** Module 02 - State Management! 🚀
